<a href="https://colab.research.google.com/github/faiyaz025/CSE221-lab-codes/blob/main/CSE221_Lab8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Task A

In [ ]:
num_nodes, num_queries = map(int, input().split())
leaders = list(range(num_nodes + 1))
component_size = [1] * (num_nodes + 1)

def get_leader(x):
    if leaders[x] != x:
        leaders[x] = get_leader(leaders[x])
    return leaders[x]

def join_sets(x, y):
    root_x = get_leader(x)
    root_y = get_leader(y)
    if root_x == root_y:
        return component_size[root_x]
    if component_size[root_x] < component_size[root_y]:
        root_x, root_y = root_y, root_x
    leaders[root_y] = root_x
    component_size[root_x] += component_size[root_y]
    return component_size[root_x]

for _ in range(num_queries):
    u, v = map(int, input().split())
    print(join_sets(u, v))

Task B

In [ ]:
def get_root(parents, node):
    if parents[node] != node:
        parents[node] = get_root(parents, parents[node])
    return parents[node]

def merge_sets(parents, heights, a, b):
    root_a = get_root(parents, a)
    root_b = get_root(parents, b)
    if root_a == root_b:
        return False
    if heights[root_a] < heights[root_b]:
        parents[root_a] = root_b
    else:
        parents[root_b] = root_a
        if heights[root_a] == heights[root_b]:
            heights[root_a] += 1
    return True

nodes, edges_count = map(int, input().split())
all_edges = []

for _ in range(edges_count):
    src, dest, weight = map(int, input().split())
    all_edges.append((weight, src, dest))

all_edges.sort()

parents = list(range(nodes + 1))
heights = [0] * (nodes + 1)

total_cost = 0
for weight, src, dest in all_edges:
    if merge_sets(parents, heights, src, dest):
        total_cost += weight

print(total_cost)

Task C

In [ ]:
class DSU:
    def __init__(self, n):
        self.parent = list(range(n))
        self.rank = [1] * n

    def find(self, i):
        if self.parent[i] != i:
            self.parent[i] = self.find(self.parent[i])
        return self.parent[i]

    def unite(self, x, y):
        rootX = self.find(x)
        rootY = self.find(y)
        if rootX == rootY:
            return False
        if self.rank[rootX] < self.rank[rootY]:
            self.parent[rootX] = rootY
        elif self.rank[rootX] > self.rank[rootY]:
            self.parent[rootY] = rootX
        else:
            self.parent[rootY] = rootX
            self.rank[rootX] += 1
        return True

def dfs_strictly_smaller(u, v, limit, adj, vis, stamp):
    stack = [(u, -1)]
    while stack:
        node, max_below = stack.pop()
        if node == v:
            return max_below
        if vis[node] == stamp:
            continue
        vis[node] = stamp
        for nei, w in adj[node]:
            if vis[nei] != stamp:
                best = max_below
                if w < limit and w > max_below:
                    best = w
                stack.append((nei, best))
    return -1

def secondMST(n, edges):
    edges.sort()
    dsu = DSU(n)
    mst_cost = 0
    adj = [[] for _ in range(n)]
    used = [False] * len(edges)

    for i, (w, u, v) in enumerate(edges):
        if dsu.unite(u, v):
            used[i] = True
            mst_cost += w
            adj[u].append((v, w))
            adj[v].append((u, w))

    if sum(used) != n - 1:
        return -1

    vis = [0] * n
    stamp = 1
    res = float('inf')

    for i, (w, u, v) in enumerate(edges):
        if used[i]:
            continue
        stamp += 1
        smaller = dfs_strictly_smaller(u, v, w, adj, vis, stamp)
        if smaller != -1:
            new_cost = mst_cost - smaller + w
            if new_cost > mst_cost:
                res = min(res, new_cost)

    return -1 if res == float('inf') else res

# Input handling
n, m = map(int, input().split())
edges = []
for _ in range(m):
    u, v, w = map(int, input().split())
    edges.append((w, u - 1, v - 1))

print(secondMST(n, edges))